# 春闘における一時金の支給状況PDFの加工
春闘における一時金は、夏季及び冬季それぞれについて、産業別及び集計組合別に、月数ベースと金額ベースで公表されている。
これを編集しやすいように時系列データに加工する。

## 夏季・年間一時金のデータ整理

In [14]:
import tabula
import pandas as pd
import json

f = json.load(open('pdf_list.json', 'r'))



In [ ]:

colnames = ['属性', '組合数', '組合人員数', '加重平均‗要求', '加重平均‗回答', '加重平均‗前年実績', '単純平均‗要求', '単純平均‗回答', '単純平均‗前年実績']


# for文で得られたdfをすべて縦に結合する為の空のdfを作成する
master_df = pd.DataFrame()


for i in f['summer']:

    pdfpath = f['summer'][i]

    dfs = tabula.read_pdf(pdfpath, 
                    pages="all",
                    lattice=True
                    )

    # dfsの中身をすべて縦に結合する
    df = pd.concat(dfs, axis=0)


    # 業種別がNaNである時、構成組織の値をとる
    df['業種別'] = df['業種別'].fillna(df['構成組織'])

    # 業種別がNaNである行を削除する
    df = df.dropna(subset=['業種別'])
    # 構成組織の列を削除する
    df = df.drop(columns=['構成組織'])

    # index列を削除してから、先頭にyear列を挿入する
    df = df.reset_index(drop=True)
    # 列名をcolnamesに変更する
    df.columns = colnames
    df.insert(0, '集計年', i[1:])

    # 加重平均‗要求列を、','を削除してからfloat型に変換する
    df['加重平均‗要求'] = df['加重平均‗要求'].str.replace(',', '').astype(float)

    # '集計年'列の一つ後ろに、加重平均‗要求列の値が100以下であれば'月数'を、そうでなければ'金額'をとる列を挿入する
    df.insert(2, '集計カテゴリ', df.apply(lambda x: '月数' if x['加重平均‗要求'] <= 100 else '金額', axis=1))

    # '属性'と'集計カテゴリ'の両方からなるグループを作って、それぞれのグループごとにソートする。その後、グループの中でindexの小さい順に数字を1から順番に振る。
    df = df.sort_values(['属性', '集計カテゴリ']).reset_index(drop=True)
    df.insert(0, 'id', df.groupby(['属性', '集計カテゴリ']).cumcount() + 1)
    # 'id'が1の時は'年間',2の時は'夏季'という文字列を'集計カテゴリ'の後ろに追加する
    df['集計カテゴリ'] = df.apply(lambda x: x['集計カテゴリ'] + '（年間）' if x['id'] == 1 else x['集計カテゴリ'] + '（夏季）', axis=1)

    df = df.drop(columns=['id'])

    # dfをdf‗masterに追加していく
    master_df = pd.concat([master_df, df], axis=0)


master_df.to_csv('../data/summer_bonus.csv', index=False, encoding='utf-8-sig')




## 冬季一時金の集計についても同様に行う

In [69]:
import tabula
import pandas as pd

colnames = ['属性', '組合数', '組合人員数', '加重平均‗回答', '加重平均‗前年実績','単純平均‗回答', '単純平均‗前年実績']


# for文で得られたdfをすべて縦に結合する為の空のdfを作成する
master_df = pd.DataFrame()


for i in f['winter']:

    pdfpath = f['winter'][i]

    dfs = tabula.read_pdf(pdfpath, 
                    pages="all",
                    lattice=True
                    )

    # dfsの中身をすべて縦に結合する
    df = pd.concat(dfs, axis=0)


    # 業種別がNaNである時、構成組織の値をとる
    df['業種別'] = df['業種別'].fillna(df['構成組織'])

    # 業種別がNaNである行を削除する
    df = df.dropna(subset=['業種別'])
    # 構成組織の列を削除する
    df = df.drop(columns=['構成組織'])    

    # もし変数の数が９個であれば,前の２つの変数を補完してからdfの後ろ２列を削除する
    if len(df.columns) == 9:
        df[df.columns[1]] = df[df.columns[1]].fillna(df[df.columns[-2]])
        df[df.columns[2]] = df[df.columns[2]].fillna(df[df.columns[-1]])
        df = df.drop(columns=[df.columns[-1], df.columns[-2]])

    # index列を削除してから、先頭にyear列を挿入する
    df = df.reset_index(drop=True)
    # 列名をcolnamesに変更する
    df.columns = colnames
    df.insert(0, '集計年', i[1:])

    # 加重平均‗回答列を、','を削除してからfloat型に変換する
    df['加重平均‗回答'] = df['加重平均‗回答'].str.replace(',', '').astype(float)

    # '集計年'列の一つ後ろに、加重平均‗要求列の値が100以下であれば'月数'を、そうでなければ'金額'をとる列を挿入する
    df.insert(2, '集計カテゴリ', df.apply(lambda x: '月数（冬季）' if x['加重平均‗回答'] <= 100 else '金額（冬季）', axis=1))

    # dfをdf‗masterに追加していく
    master_df = pd.concat([master_df, df], axis=0)

master_df.to_csv('../data/winter_bonus.csv', index=False, encoding='utf-8_sig')



Nov 14, 2023 4:56:54 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Nov 14, 2023 4:56:54 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Nov 14, 2023 4:56:55 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Nov 14, 2023 4:56:55 AM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Nov 14, 2023 4:56:55 AM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Nov 14, 2023 4:56:55 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Nov 14, 2023 4:56:55 AM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Nov 14, 2023 4:56:55 AM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Nov 14, 2023 4:56:55 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Nov 14, 2023 4:56:55 AM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Nov 14, 2023 4:56:55 AM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Nov 14, 2023 4:56:55 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Nov 14, 2023 4:56:55 AM org.apache.fontbox.ttf.CmapSubtable processSubtype14